In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely import wkt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sqlalchemy import create_engine, text
import geopy.distance
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
signals_geo = gpd.read_file('../data/traffic_signals_geo.zip')

In [4]:
devices_geo = gpd.read_file('../data/intelligent_traffic_system_devices.zip')

In [5]:
accidents_geo = pd.read_csv('../data/traffic_accidents.csv')

C:\Users\MattM\AppData\Local\Temp\ipykernel_4124\3003402844.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents_geo = pd.read_csv('../data/traffic_accidents.csv')


In [22]:
accidents_geo.head()

,offense_id,dist_from_signal,top_traffic_accident_offense,reported_date,incident_address,lon,lat,district_id,precinct_id,neighborhood_id,bicycle_ind,pedestrian_ind,HARMFUL_EVENT_SEQ_MOST,ROAD_DESCRIPTION,ROAD_CONDITION,LIGHT_CONDITION,TU1_VEHICLE_TYPE,TU1_TRAVEL_DIRECTION,TU1_VEHICLE_MOVEMENT,TU1_DRIVER_ACTION,TU1_DRIVER_HUMANCONTRIBFACTOR,TU1_PEDESTRIAN_ACTION,TU2_VEHICLE_TYPE,TU2_TRAVEL_DIRECTION,TU2_VEHICLE_MOVEMENT,TU2_DRIVER_ACTION,TU2_DRIVER_HUMANCONTRIBFACTOR,TU2_PEDESTRIAN_ACTION,SERIOUSLY_INJURED,FATALITIES,FATALITY_MODE_1,FATALITY_MODE_2,SERIOUSLY_INJURED_MODE_1,SERIOUSLY_INJURED_MODE_2,POINT_X,POINT_Y,geometry,offense_clean,month,month_name,year,day_name,hour,zipcode,season,dist_from_camera,feet_from_signal,feet_from_camera,bin_from_signal
0,20134854410,0.000106,TRAF - ACCIDENT,1/1/2013 0:19,W COLFAX AVE / N BROADWAY ST,-104.987391,39.740059,6,611.0,Civic Center,0.0,0.0,GUARD RAIL,non intersection,dry,dark lighted,PASSENGER CAR/VAN,east,GOING STRAIGHT,CARELESS DRIVING,DISTRACTED PASSENGER,,,,,,,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9873914 39.7400591),ACCIDENT,1,January,2013,Tuesday,0,80202,WINTER,0.000106,0.55968,0.55968,0.0
1,20138854410,0.009364,TRAF - ACCIDENT,1/1/2013 0:32,2300 N BROADWAY ST,-104.987293,39.754540,6,621.0,Five Points,0.0,0.0,FRONT TO SIDE WITH MV IN TRANSPORT,at intersection,dry,dark lighted,SUV,north,MAKING LEFT TURN,FAILED TO YIELD ROW,OTHER,,PASSENGER CAR/VAN,SOUTH,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9872927 39.75453992),ACCIDENT,1,January,2013,Tuesday,0,80205,WINTER,0.111080,49.44192,586.50240,0.0
2,2013354010,0.000141,TRAF - ACCIDENT - HIT & RUN,1/1/2013 0:33,N WASHINGTON ST / RINGSBY CT,-104.978825,39.775885,1,112.0,Globeville,0.0,0.0,FRONT TO SIDE WITH MV IN TRANSPORT,at intersection,dry,dark lighted,PASSENGER CAR/VAN,east,MAKING U-TURN,FAILED TO YIELD ROW,NO APPARENT,OTHER,SUV,NORTH,GOING STRAIGHT,OTHER,NO APPARENT,OTHER,0.0,0.0,,,,,NaN,NaN,POINT (-104.9788255 39.77588543),HIT & RUN,1,January,2013,Tuesday,0,80216,WINTER,0.073015,0.74448,385.51920,0.0
3,201310954410,0.000166,TRAF - ACCIDENT,1/1/2013 0:45,N UNIVERSITY BLVD / E 1ST AVE,-104.959222,39.718275,3,311.0,Cherry Creek,0.0,0.0,FRONT TO SIDE WITH MV IN TRANSPORT,at intersection,dry,dark lighted,SUV,east,GOING STRAIGHT,DISREGARDED OTHER DEVICE,NO APPARENT,,PASSENGER CAR/VAN,NORTH,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9592218 39.71827547),ACCIDENT,1,January,2013,Tuesday,0,80206,WINTER,0.000164,0.87648,0.86592,0.0
4,201312954410,0.023657,TRAF - ACCIDENT,1/1/2013 1:03,1200 BLOCK N BROADWAY ST,-104.987055,39.735476,6,623.0,Capitol Hill,0.0,1.0,PEDESTRIAN-OTHER,non intersection,dry,dark unlighted,,west,,,,CROSS/ENTER NOT AT INTERSECTION,,SOUTH,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9870551 39.73547578),ACCIDENT,1,January,2013,Tuesday,1,80203,WINTER,0.106728,124.90896,563.52384,0.0


In [16]:
def bin_signal_dist(x):
    if 0 <= x['dist_from_signal'] <=0.09:
        b = '0.0'
    elif .1 <= x['dist_from_signal'] <=0.19:
        b = '0.1'
    elif .2 <= x['dist_from_signal'] <=0.29:
        b = '0.2'
    elif .3 <= x['dist_from_signal'] <=0.39:
        b = '0.3'
    elif .4 <= x['dist_from_signal'] <=0.49:
        b = '0.4'
    elif .5 <= x['dist_from_signal'] <=0.59:
        b = '0.5'
    elif .6 <= x['dist_from_signal'] <=0.69:
        b = '0.6'
    elif .7 <= x['dist_from_signal'] <=0.79:
        b = '0.7'
    elif .8 <= x['dist_from_signal'] <=0.89:
        b = '0.8'
    elif .9 <= x['dist_from_signal'] <=0.99:
        b = '0.9'
    elif 1 <= x['dist_from_signal']:
        b = '1.0+'
    else:
        b = 'ERROR'
    return b
    


In [26]:
accidents_geo = accidents_geo.assign(bin_from_signal=accidents_geo.apply(bin_signal_dist, axis=1))

In [23]:
def bin_camera_dist(x):
    if 0 <= x['dist_from_camera'] <=0.09:
        b = '0.0'
    elif .1 <= x['dist_from_camera'] <=0.19:
        b = '0.1'
    elif .2 <= x['dist_from_camera'] <=0.29:
        b = '0.2'
    elif .3 <= x['dist_from_camera'] <=0.39:
        b = '0.3'
    elif .4 <= x['dist_from_camera'] <=0.49:
        b = '0.4'
    elif .5 <= x['dist_from_camera'] <=0.59:
        b = '0.5'
    elif .6 <= x['dist_from_camera'] <=0.69:
        b = '0.6'
    elif .7 <= x['dist_from_camera'] <=0.79:
        b = '0.7'
    elif .8 <= x['dist_from_camera'] <=0.89:
        b = '0.8'
    elif .9 <= x['dist_from_camera'] <=0.99:
        b = '0.9'
    elif 1 <= x['dist_from_camera']:
        b = '1.0+'
    else:
        b = 'ERROR'
    return b

In [28]:
accidents_geo = accidents_geo.assign(bin_from_camera=accidents_geo.apply(bin_camera_dist, axis=1))

In [29]:
accidents_geo.head()

,offense_id,dist_from_signal,top_traffic_accident_offense,reported_date,incident_address,lon,lat,district_id,precinct_id,neighborhood_id,bicycle_ind,pedestrian_ind,HARMFUL_EVENT_SEQ_MOST,ROAD_DESCRIPTION,ROAD_CONDITION,LIGHT_CONDITION,TU1_VEHICLE_TYPE,TU1_TRAVEL_DIRECTION,TU1_VEHICLE_MOVEMENT,TU1_DRIVER_ACTION,TU1_DRIVER_HUMANCONTRIBFACTOR,TU1_PEDESTRIAN_ACTION,TU2_VEHICLE_TYPE,TU2_TRAVEL_DIRECTION,TU2_VEHICLE_MOVEMENT,TU2_DRIVER_ACTION,TU2_DRIVER_HUMANCONTRIBFACTOR,TU2_PEDESTRIAN_ACTION,SERIOUSLY_INJURED,FATALITIES,FATALITY_MODE_1,FATALITY_MODE_2,SERIOUSLY_INJURED_MODE_1,SERIOUSLY_INJURED_MODE_2,POINT_X,POINT_Y,geometry,offense_clean,month,month_name,year,day_name,hour,zipcode,season,dist_from_camera,feet_from_signal,feet_from_camera,bin_from_signal,bin_from_camera
0,20134854410,0.000106,TRAF - ACCIDENT,1/1/2013 0:19,W COLFAX AVE / N BROADWAY ST,-104.987391,39.740059,6,611.0,Civic Center,0.0,0.0,GUARD RAIL,non intersection,dry,dark lighted,PASSENGER CAR/VAN,east,GOING STRAIGHT,CARELESS DRIVING,DISTRACTED PASSENGER,,,,,,,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9873914 39.7400591),ACCIDENT,1,January,2013,Tuesday,0,80202,WINTER,0.000106,0.55968,0.55968,0.0,0.0
1,20138854410,0.009364,TRAF - ACCIDENT,1/1/2013 0:32,2300 N BROADWAY ST,-104.987293,39.754540,6,621.0,Five Points,0.0,0.0,FRONT TO SIDE WITH MV IN TRANSPORT,at intersection,dry,dark lighted,SUV,north,MAKING LEFT TURN,FAILED TO YIELD ROW,OTHER,,PASSENGER CAR/VAN,SOUTH,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9872927 39.75453992),ACCIDENT,1,January,2013,Tuesday,0,80205,WINTER,0.111080,49.44192,586.50240,0.0,0.1
2,2013354010,0.000141,TRAF - ACCIDENT - HIT & RUN,1/1/2013 0:33,N WASHINGTON ST / RINGSBY CT,-104.978825,39.775885,1,112.0,Globeville,0.0,0.0,FRONT TO SIDE WITH MV IN TRANSPORT,at intersection,dry,dark lighted,PASSENGER CAR/VAN,east,MAKING U-TURN,FAILED TO YIELD ROW,NO APPARENT,OTHER,SUV,NORTH,GOING STRAIGHT,OTHER,NO APPARENT,OTHER,0.0,0.0,,,,,NaN,NaN,POINT (-104.9788255 39.77588543),HIT & RUN,1,January,2013,Tuesday,0,80216,WINTER,0.073015,0.74448,385.51920,0.0,0.0
3,201310954410,0.000166,TRAF - ACCIDENT,1/1/2013 0:45,N UNIVERSITY BLVD / E 1ST AVE,-104.959222,39.718275,3,311.0,Cherry Creek,0.0,0.0,FRONT TO SIDE WITH MV IN TRANSPORT,at intersection,dry,dark lighted,SUV,east,GOING STRAIGHT,DISREGARDED OTHER DEVICE,NO APPARENT,,PASSENGER CAR/VAN,NORTH,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9592218 39.71827547),ACCIDENT,1,January,2013,Tuesday,0,80206,WINTER,0.000164,0.87648,0.86592,0.0,0.0
4,201312954410,0.023657,TRAF - ACCIDENT,1/1/2013 1:03,1200 BLOCK N BROADWAY ST,-104.987055,39.735476,6,623.0,Capitol Hill,0.0,1.0,PEDESTRIAN-OTHER,non intersection,dry,dark unlighted,,west,,,,CROSS/ENTER NOT AT INTERSECTION,,SOUTH,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,,NaN,NaN,POINT (-104.9870551 39.73547578),ACCIDENT,1,January,2013,Tuesday,1,80203,WINTER,0.106728,124.90896,563.52384,0.0,0.1


In [31]:
accidents_geo.to_csv('../data/traffic_accidents.csv', index=False)

In [42]:
bins = accidents_geo.groupby('bin_from_camera').agg({'offense_clean':'value_counts'})

In [38]:
bin_totals = accidents_geo.groupby('bin_from_camera').agg({'offense_clean':'count'}).reset_index()

In [39]:
bin_totals.head()

,bin_from_camera,offense_clean
0,0.0,101958
1,0.1,41341
2,0.2,24656
3,0.3,10908
4,0.4,6849


In [47]:
bins = bins.rename(columns = {'bin_from_camera':'bin', 'offense_clean':'offense' ,'offense_clean':'offense_count'}).reset_index()

In [51]:
bins

,bin_from_camera,offense_clean_x,offense_count,offense_clean_y
0,0.0,ACCIDENT,69204,101958
1,0.0,HIT & RUN,27180,101958
2,0.0,DUI/DUID,2676,101958
3,0.0,SBI,1674,101958
4,0.0,POLICE,930,101958
...,...,...,...,...
67,ERROR,HIT & RUN,4039,12720
68,ERROR,DUI/DUID,369,12720
69,ERROR,SBI,205,12720
70,ERROR,POLICE,198,12720


In [50]:
bins = pd.merge(bins, bin_totals, on='bin_from_camera')

In [52]:
bins = bins.rename(columns = {'bin_from_camera':'bin', 'offense_clean_x':'offense', 'offense_count':'count','offense_clean_y':'total'})
bins.head()

,bin,offense,count,total
0,0.0,ACCIDENT,69204,101958
1,0.0,HIT & RUN,27180,101958
2,0.0,DUI/DUID,2676,101958
3,0.0,SBI,1674,101958
4,0.0,POLICE,930,101958


In [54]:
bins['offense_%'] = bins['count'] /bins['total'] *100
bins

,bin,offense,count,total,offense_%
0,0.0,ACCIDENT,69204,101958,67.875007
1,0.0,HIT & RUN,27180,101958,26.658036
2,0.0,DUI/DUID,2676,101958,2.624610
3,0.0,SBI,1674,101958,1.641853
4,0.0,POLICE,930,101958,0.912140
...,...,...,...,...,...
67,ERROR,HIT & RUN,4039,12720,31.753145
68,ERROR,DUI/DUID,369,12720,2.900943
69,ERROR,SBI,205,12720,1.611635
70,ERROR,POLICE,198,12720,1.556604


In [55]:
camera_perc = bins

In [56]:
camera_perc.to_csv('../data/camera_perc.csv', index=False)

In [57]:
bins = accidents_geo.groupby('bin_from_signal').agg({'offense_clean':'value_counts'})

In [58]:
bin_totals = accidents_geo.groupby('bin_from_signal').agg({'offense_clean':'count'}).reset_index()

In [59]:
bin_totals.head()

,bin_from_signal,offense_clean
0,0.0,139202
1,0.1,34247
2,0.2,14486
3,0.3,5500
4,0.4,1951


In [60]:
bins = bins.rename(columns = {'bin_from_signal':'bin', 'offense_clean':'offense' ,'offense_clean':'offense_count'}).reset_index()

In [61]:
bins

,bin_from_signal,offense_clean,offense_count
0,0.0,ACCIDENT,93809
1,0.0,HIT & RUN,37747
2,0.0,DUI/DUID,3640
3,0.0,SBI,2304
4,0.0,POLICE,1325
...,...,...,...
67,ERROR,HIT & RUN,3125
68,ERROR,DUI/DUID,268
69,ERROR,SBI,133
70,ERROR,POLICE,98


In [62]:
bins = pd.merge(bins, bin_totals, on='bin_from_signal')

In [63]:
bins = bins.rename(columns = {'bin_from_signal':'bin', 'offense_clean_x':'offense', 'offense_count':'count','offense_clean_y':'total'})
bins.head()

,bin,offense,count,total
0,0.0,ACCIDENT,93809,139202
1,0.0,HIT & RUN,37747,139202
2,0.0,DUI/DUID,3640,139202
3,0.0,SBI,2304,139202
4,0.0,POLICE,1325,139202


In [64]:
bins['offense_%'] = bins['count'] /bins['total'] *100
bins

,bin,offense,count,total,offense_%
0,0.0,ACCIDENT,93809,139202,67.390555
1,0.0,HIT & RUN,37747,139202,27.116708
2,0.0,DUI/DUID,3640,139202,2.614905
3,0.0,SBI,2304,139202,1.655149
4,0.0,POLICE,1325,139202,0.951854
...,...,...,...,...,...
67,ERROR,HIT & RUN,3125,9389,33.283630
68,ERROR,DUI/DUID,268,9389,2.854404
69,ERROR,SBI,133,9389,1.416551
70,ERROR,POLICE,98,9389,1.043775


In [65]:
signal_perc = bins

In [66]:
signal_perc.to_csv('../data/signal_perc.csv', index=False)

In [71]:
signals_geo

,TRAFFIC_SI,RECORD_DAT,RECORD_STA,SIGNAL_STA,FROM_DATE,TO_DATE,SIGNAL_OWN,LOCATION_C,LOCATION_I,SIGNAL_TYP,INSTALLATI,REBUILD_DA,MAJOR_RETI,geometry
0,171,2012-01-09,ACTIVE,IN USE,2012-01-01,None,CCD,4422,658,None,None,None,None,POINT (-105.00081 39.74669)
1,172,2012-01-09,ACTIVE,IN USE,2012-01-01,None,CCD,3085,63,None,None,None,None,POINT (-105.00142 39.74641)
2,173,2012-01-09,ACTIVE,IN USE,2012-01-01,None,CCD,4020,504,None,None,None,None,POINT (-104.99404 39.72893)
3,174,2012-01-09,ACTIVE,IN USE,2012-01-01,None,CCD,3292N,213,None,None,None,None,POINT (-105.00018 39.72892)
4,176,2012-01-09,ACTIVE,IN USE,2012-01-01,None,CCD,3975,489,None,None,None,None,POINT (-104.98038 39.71112)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,1287,2012-11-19,ACTIVE,IN USE,2012-11-19,None,CCD,6603,0,None,None,None,None,POINT (-104.89374 39.66760)
1283,1291,2012-11-19,ACTIVE,IN USE,2012-11-19,None,CCD,3752,0,None,None,None,None,POINT (-104.95916 39.74162)
1284,1295,2012-11-19,ACTIVE,IN USE,2012-11-19,None,CCD,6751,0,None,None,None,None,POINT (-104.88634 39.77686)
1285,1290,2012-11-19,ACTIVE,IN USE,2012-11-19,None,CCD,5918,0,None,None,None,None,POINT (-104.95978 39.74162)


In [70]:
signals_geo['SIGNAL_TYP'].value_counts()

Series([], Name: count, dtype: int64)

In [72]:
devices_geo

,ITS_DEVICE,ITS_DEVI_1,RECORD_DAT,ITS_DEVI_2,LOCATION_D,ITS_DEVI_3,ITS_DEVI_4,CAMERA_VIE,EDIT_REASO,PUBLISHED_,POLE_LOCAT,LOCATION_O,geometry
0,626,TRAFFICCAM,2016-02-25,IN USE,Brighton and 31st,538,PWTES,PTZ,2,None,SE Corner,Brighton and 31st,POINT (-104.98367 39.76641)
1,627,TRAFFICCAM,2016-02-28,IN USE,Brighton and 44th,539,PWTES,PTZ,2,None,NW Corner,Brighton and 44th,POINT (-104.96842 39.77853)
2,628,TRAFFICCAM,2016-03-13,IN USE,Colfax and Wolff,540,PWTES,PTZ,2,None,SE Corner,Colfax and Wolff,POINT (-105.04857 39.74025)
3,146,Traffic camera,2012-01-09,"Accepted, in service",15th and Welton,321,CCD Traffic Engineering,PTZ,0,FALSE,NE Corner,15THWELTON,POINT (-104.99252 39.74358)
4,147,Traffic camera,2012-01-09,"Accepted, in service",15th and Larimer,325,CCD Traffic Engineering,PTZ,0,FALSE,NW Corner,None,POINT (-104.99864 39.74829)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,576,Traffic camera,2014-01-01,"Accepted, in service",Havana and Girard,499,CCD Traffic Engineering,PTZ,2,None,None,None,POINT (-104.86651 39.65700)
646,577,Traffic camera,2014-01-01,"Accepted, in service",17th and Esplanade,500,CCD Traffic Engineering,PTZ,2,None,None,None,POINT (-104.95685 39.74382)
647,578,Traffic camera,2014-01-01,"Accepted, in service",Havana and Smith Rd,501,CCD Traffic Engineering,PTZ,2,None,None,None,POINT (-104.86579 39.76851)
648,579,Traffic camera,2014-01-01,"Accepted, in service",Evans and Oneida,502,CCD Traffic Engineering,PTZ,2,None,None,None,POINT (-104.90820 39.67847)
